# Task 1 - CNN Model

---
## Import

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers, losses, Input
from keras_tuner import RandomSearch, Objective
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import sys
sys.path.append('../')

from utils.tensorflow_preprocessing import prepare_image_target_dataset
from keras.callbacks import EarlyStopping, ProgbarLogger

In [2]:
# %store -r train_X
# %store -r val_X
# %store -r test_X

---
## Prepare dataset

In [3]:
save_path = "../data/processed/csv/"

train_X = pd.read_csv(save_path + 'train_X.csv')
val_X = pd.read_csv(save_path + 'val_X.csv')
test_X = pd.read_csv(save_path + 'test_X.csv')

In [4]:
train_class_ds, class_encoder = prepare_image_target_dataset(train_X, target_name="Style")
val_class_ds, _ = prepare_image_target_dataset(val_X, target_name="Style", label_encoder=class_encoder)
test_class_ds, _ = prepare_image_target_dataset(test_X, target_name="Style", label_encoder=class_encoder)

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_class_ds = train_class_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_class_ds = val_class_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_class_ds = test_class_ds.cache().prefetch(buffer_size=AUTOTUNE)

---
## Model training

In [6]:
# Basic Model Training
def create_model():
    model = models.Sequential([
        Input(shape=(256, 256, 3)),
        layers.Conv2D(4, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(8, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(16, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (4, 4), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (4, 4), activation='relu'),
        layers.GlobalMaxPooling2D(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5, seed=21),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5, seed=21),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5, seed=42),
        layers.Dense(128, activation='relu'),
        layers.Dense(17, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss=losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    return model


In [ ]:
# Train the model
early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
CNN_model = create_model()
epochs = 50
history = CNN_model.fit(train_class_ds, epochs=epochs, batch_size=32, validation_data=val_class_ds, callbacks=[early_stopping, ProgbarLogger()])



Epoch 1/50




In [ ]:
# Evaluate the model
test_loss, test_accuracy = CNN_model.evaluate(test_class_ds)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# Visualize the training history of the basic model
def plot_training_history(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training History')
    plt.show()

plot_training_history(history)

---
## Optimization

In [ ]:
# Hyperparameter Tuning
def build_model(hp):
    model = models.Sequential([
        Input(shape=(256, 256, 3)),
        layers.Conv2D(4, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(8, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(16, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (4, 4), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (4, 4), activation='relu'),
        layers.GlobalMaxPooling2D(),
        layers.Dense(hp.Choice('dense1_units', [128, 256]), activation='relu'),
        layers.Dropout(hp.Choice('dropout1_rate', [0.3, 0.5])),
        layers.Dense(hp.Choice('dense1_units', [64, 128]), activation='relu'),
        layers.Dropout(hp.Choice('dropout1_rate', [0.3, 0.5])),
        layers.Dense(hp.Choice('dense2_units', [32, 64, 128]), activation='relu'),
        layers.Dropout(hp.Choice('dropout2_rate', [0.3, 0.5])),
        layers.Dense(128, activation='relu'),
        layers.Dense(17, activation='sigmoid')
    ])

    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
                  loss=losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective=Objective('val_accuracy', direction='max'),
    max_trials=4,
    executions_per_trial=1,
    directory='tuner_results',
    project_name='basic_tuning'
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(train_class_ds,
             epochs=10,
             validation_data=val_class_ds)

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(test_class_ds)
print(f'Best Model Test Loss: {test_loss:.4f}')
print(f'Best Model Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# Compare the performance of the basic model and the best model
models = ['Basic Model', 'Tuned Model']
accuracies = [test_accuracy, test_accuracy]

plt.figure(figsize=(8, 6))
plt.bar(models, accuracies)
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.show()

# Visualize the confusion matrix
def plot_confusion_matrix(model, test_data, label_encoder):
    y_true = []
    y_pred = []
    for images, labels in test_data:
        y_true.extend(labels.numpy())
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))

    cm = confusion_matrix(y_true, y_pred)

    # Get the decoded class labels
    class_labels = label_encoder.inverse_transform(np.unique(y_true))

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    
plot_confusion_matrix(best_model, test_class_ds,class_encoder)

---
## Testing

In [ ]:
def visualize_predictions(model, test_data, label_encoder, num_samples=5):
    for images, labels in test_data.take(1):
        predictions = model.predict(images)
        predicted_labels = np.argmax(predictions, axis=1)

        plt.figure(figsize=(15, 12))
        for i in range(num_samples):
            plt.subplot(1, num_samples, i+1)
            plt.imshow((images[i].numpy() * 255).astype('uint8'))
            true_label = label_encoder.inverse_transform([labels[i]])[0]
            pred_label = label_encoder.inverse_transform([predicted_labels[i]])[0]
            plt.title(f"True: {true_label}, Pred: {pred_label}")
            plt.axis('off')
        plt.tight_layout()
        plt.show()

visualize_predictions(best_model, test_class_ds, class_encoder)

---
## Save model

In [ ]:
from pathlib import Path

# Create paths
path = '../data/models'
processed_model_dir = Path(path)
processed_model_dir.mkdir(parents=True, exist_ok=True)

best_model.save(path + '/' + 'task-3-CNN.keras')